In [1]:
pip install pyspark


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install graphframes

  Using cached graphframes-0.6-py2.py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [4]:
import findspark
findspark.init()

In [6]:
from pyspark import SparkContext,SQLContext
from graphframes import *
import pyspark.sql.functions as f

sc = SparkContext.getOrCreate()
sc.addPyFile('C:\spark-3.0.0-preview2-bin-hadoop2.7\jars\graphframes_graphframes-0.8.0-spark3.0-s_2.12.jar')
sqlcontext=SQLContext(sc)

In [8]:
station_data_df= sqlcontext.read.format("csv").option("header", "true").csv(r'C:\Users\sarik\PycharmProjects\BDP_ICP12\station_data.csv')
trips_df= sqlcontext.read.format("csv").option("header", "true").csv(r'C:\Users\sarik\PycharmProjects\BDP_ICP12\trip_data.csv')

In [9]:
station_data_df.show()

+----------+--------------------+---------+-----------+---------+------------+------------+
|station_id|                name|      lat|       long|dockcount|    landmark|installation|
+----------+--------------------+---------+-----------+---------+------------+------------+
|         2|San Jose Diridon ...|37.329732|-121.901782|       27|    San Jose|    8/6/2013|
|         3|San Jose Civic Ce...|37.330698|-121.888979|       15|    San Jose|    8/5/2013|
|         4|Santa Clara at Al...|37.333988|-121.894902|       11|    San Jose|    8/6/2013|
|         5|    Adobe on Almaden|37.331415|  -121.8932|       19|    San Jose|    8/5/2013|
|         6|    San Pedro Square|37.336721|-121.894074|       15|    San Jose|    8/7/2013|
|         7|Paseo de San Antonio|37.333798|-121.886943|       15|    San Jose|    8/7/2013|
|         8| San Salvador at 1st|37.330165|-121.885831|       15|    San Jose|    8/5/2013|
|         9|           Japantown|37.348742|-121.894715|       15|    San Jose|  

In [10]:
station_data_vertices=station_data_df.withColumnRenamed("name","id").select("id").distinct()
station_data_vertices.show()

+--------------------+
|                  id|
+--------------------+
|San Jose Diridon ...|
|San Jose Civic Ce...|
|Santa Clara at Al...|
|    Adobe on Almaden|
|    San Pedro Square|
|Paseo de San Antonio|
| San Salvador at 1st|
|           Japantown|
|  San Jose City Hall|
|         MLK Library|
|SJSU 4th at San C...|
|       St James Park|
|Arena Green / SAP...|
|SJSU - San Salvad...|
|   Franklin at Maple|
|Redwood City Calt...|
|San Mateo County ...|
|Redwood City Publ...|
|Stanford in Redwo...|
|Redwood City Medi...|
+--------------------+
only showing top 20 rows



In [11]:
trip_edges=trips_df.withColumnRenamed("Start Station","src")\
.withColumnRenamed("End Station","dst")\
.select("src","dst")

In [12]:
trip_edges.show()

+--------------------+--------------------+
|                 src|                 dst|
+--------------------+--------------------+
|Harry Bridges Pla...|San Francisco Cal...|
|San Antonio Shopp...|Mountain View Cit...|
|      Post at Kearny|   2nd at South Park|
|  San Jose City Hall| San Salvador at 1st|
|Embarcadero at Fo...|Embarcadero at Sa...|
|Yerba Buena Cente...|San Francisco Cal...|
|Embarcadero at Fo...|Embarcadero at Sa...|
|Embarcadero at Sa...|   Steuart at Market|
|     Beale at Market|Temporary Transba...|
|      Post at Kearny|South Van Ness at...|
|Embarcadero at Sa...|   Market at Sansome|
|      Market at 10th|San Francisco Cal...|
|       Market at 4th|Grant Avenue at C...|
|   Market at Sansome|Broadway St at Ba...|
|Temporary Transba...|San Francisco Cal...|
|San Francisco Cal...|     Townsend at 7th|
|University and Em...|Cowper at University|
|     Spear at Folsom|San Francisco Cal...|
|Temporary Transba...|Grant Avenue at C...|
|San Francisco Cal...|Broadway S

In [13]:
g=GraphFrame(station_data_vertices,trip_edges)
g

GraphFrame(v:[id: string], e:[src: string, dst: string])

In [14]:
g.triangleCount().show()

+-----+--------------------+
|count|                  id|
+-----+--------------------+
|  496|       2nd at Folsom|
|   23|California Ave Ca...|
|    0|Washington at Kea...|
|  496|Powell at Post (U...|
|  496| Golden Gate at Polk|
|  496|Yerba Buena Cente...|
|  496|   Market at Sansome|
|   90|         MLK Library|
|  496|     Spear at Folsom|
|   77|           Japantown|
|  496|Commercial at Mon...|
|   81|Paseo de San Antonio|
|   23|Rengstorff Avenue...|
|   61| San Salvador at 1st|
|  496|     Townsend at 7th|
|  496|Civic Center BART...|
|   41|         Ryland Park|
|   90|San Jose Diridon ...|
|   63|San Jose Civic Ce...|
|    0|     Post at Kearney|
+-----+--------------------+
only showing top 20 rows



In [15]:
g.shortestPaths(landmarks=["2nd at Folsom", "Japantown"]).show()

+--------------------+--------------------+
|                  id|           distances|
+--------------------+--------------------+
|San Jose Diridon ...|    [Japantown -> 1]|
|Embarcadero at Va...|[2nd at Folsom -> 2]|
|    San Pedro Square|    [Japantown -> 1]|
|Arena Green / SAP...|    [Japantown -> 1]|
|San Francisco Cal...|[2nd at Folsom -> 1]|
|Washington at Kea...|                  []|
|San Francisco Cal...|[2nd at Folsom -> 1]|
|   Steuart at Market|[2nd at Folsom -> 1]|
|Broadway St at Ba...|[2nd at Folsom -> 1]|
|         Ryland Park|    [Japantown -> 1]|
|   Market at Sansome|[2nd at Folsom -> 1]|
|    Davis at Jackson|[2nd at Folsom -> 1]|
|San Mateo County ...|                  []|
|Redwood City Calt...|                  []|
|         MLK Library|    [Japantown -> 1]|
|   Franklin at Maple|                  []|
|Mountain View Cal...|                  []|
|Grant Avenue at C...|[2nd at Folsom -> 1]|
| Golden Gate at Polk|[2nd at Folsom -> 2]|
|San Antonio Shopp...|          

In [16]:
result = g.pageRank(resetProbability=0.15, tol=0.01)
result.vertices.show()
result.edges.show()

+--------------------+-------------------+
|                  id|           pagerank|
+--------------------+-------------------+
|San Jose Diridon ...|  3.895584402131728|
|Embarcadero at Va...| 0.7795867999185692|
|    San Pedro Square| 1.8004572705485624|
|Arena Green / SAP...| 0.4537065997168693|
|San Francisco Cal...|   2.57164476974859|
|Washington at Kea...|0.16889626936497573|
|San Francisco Cal...| 2.8939099915923805|
|   Steuart at Market|  1.666648284594275|
|Broadway St at Ba...| 0.6542773822104674|
|         Ryland Park| 0.8388444854882504|
|   Market at Sansome| 1.3054243296675343|
|    Davis at Jackson| 0.7741963042010769|
|San Mateo County ...| 0.9097924987564018|
|Redwood City Calt...|  1.534463751106983|
|         MLK Library| 0.7773824929442177|
|   Franklin at Maple|0.49249612010861465|
|Mountain View Cal...|  2.957150714285198|
|Grant Avenue at C...| 0.5752670064570555|
| Golden Gate at Polk| 0.4621083415369985|
|San Antonio Shopp...| 0.8947155902287497|
+----------

In [17]:
g.vertices.write.parquet('vertices')
g.edges.write.parquet('edges')

In [18]:
result = g.labelPropagation(maxIter=5)
result.select("id", "label").show()

+--------------------+-------------+
|                  id|        label|
+--------------------+-------------+
|San Jose Diridon ...|1365799600128|
|Embarcadero at Va...|1657857376256|
|    San Pedro Square| 292057776128|
|Arena Green / SAP...|1365799600128|
|San Francisco Cal...|1657857376256|
|Washington at Kea...|  17179869184|
|San Francisco Cal...|1657857376256|
|   Steuart at Market|1657857376256|
|Broadway St at Ba...|1657857376256|
|         Ryland Park| 292057776128|
|   Market at Sansome|1657857376256|
|    Davis at Jackson|1657857376256|
|San Mateo County ...| 730144440320|
|Redwood City Calt...|1151051235328|
|         MLK Library| 292057776128|
|   Franklin at Maple| 730144440320|
|Mountain View Cal...| 463856467969|
|Grant Avenue at C...|1657857376256|
| Golden Gate at Polk|1657857376256|
|San Antonio Shopp...| 661424963584|
+--------------------+-------------+
only showing top 20 rows



In [19]:
paths = g.bfs("id = 'Japantown'", "id = 'MLK Library'")
paths.show()

+-----------+--------------------+-------------+
|       from|                  e0|           to|
+-----------+--------------------+-------------+
|[Japantown]|[Japantown, MLK L...|[MLK Library]|
|[Japantown]|[Japantown, MLK L...|[MLK Library]|
|[Japantown]|[Japantown, MLK L...|[MLK Library]|
|[Japantown]|[Japantown, MLK L...|[MLK Library]|
|[Japantown]|[Japantown, MLK L...|[MLK Library]|
|[Japantown]|[Japantown, MLK L...|[MLK Library]|
|[Japantown]|[Japantown, MLK L...|[MLK Library]|
+-----------+--------------------+-------------+

